# Notebook Setup

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import sklearn as sk

In [90]:
df = pd.read_csv('../data/clean-data/cleaned_nys_data.csv')
df_original = df.copy()
df.info()

C:\Users\erin\AppData\Local\Temp\ipykernel_33092\1206858015.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/clean-data/cleaned_nys_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233144 entries, 0 to 233143
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   action_taken                       233144 non-null  int64  
 1   derived_race                       233144 non-null  object 
 2   derived_ethnicity                  233144 non-null  object 
 3   applicant_sex                      233144 non-null  int64  
 4   applicant_age                      233144 non-null  object 
 5   income                             226538 non-null  float64
 6   debt_to_income_ratio               100684 non-null  float64
 7   applicant_credit_score_type        233144 non-null  int64  
 8   loan_amount                        233144 non-null  float64
 9   loan_to_value_ratio                224447 non-null  float64
 10  interest_rate                      176143 non-null  float64
 11  rate_spread                        1519

# Impact Analysis Using HMDA Loan Approval Data, proxy outcome
regression models (MLR, logistic regression, ML algorithms?) – Victor, Zheka
Keeping predictors similar but testing across different outcome variables (loan amount, interest rate, application outcome)
Switching zip-code with race and testing other outcomes (?)

Model 1: logistic regression that predicts loan outcome
Model 2: loan amount (MLR)
Model 3: interest rate (MLR), rate_spread

In [93]:
df_slice = df.loc[(df['derived_race'] == "Black or African American") | (df['derived_race'] == "White")]
df_race = df_slice.copy()
df_race['binary_race'] = 0
df_race.loc[df_race['derived_race'] == "White", 'binary_race'] = 1

# # transform derived ethnicity to binary column for Hispanic or not
df_race['binary_ethnicity'] = 0
df_race.loc[df_race['derived_ethnicity'] == "Not Hispanic or Latino", 'binary_ethnicity'] = 1

# # create column with 0 for denied applications/preapproval requests and 1 for accepted applications
df_race['binary_denied'] = 0
df_race.loc[(df_race['action_taken'] == 3) | (df_race['action_taken'] == 7), 'binary_denied'] = 1

# # create correlation matrix
df_corr = df_race[['applicant_sex','binary_race','binary_ethnicity','binary_denied']]
df_corr.corr()

,applicant_sex,binary_race,binary_ethnicity,binary_denied
applicant_sex,1.000000,-0.089703,-0.000775,0.021091
binary_race,-0.089703,1.000000,-0.000189,-0.124504
binary_ethnicity,-0.000775,-0.000189,1.000000,-0.063217
binary_denied,0.021091,-0.124504,-0.063217,1.000000


In [94]:
df.columns

Index(['action_taken', 'derived_race', 'derived_ethnicity', 'applicant_sex',
       'applicant_age', 'income', 'debt_to_income_ratio',
       'applicant_credit_score_type', 'loan_amount', 'loan_to_value_ratio',
       'interest_rate', 'rate_spread', 'loan_type', 'loan_purpose',
       'lien_status', 'property_value', 'occupancy_type',
       'tract_minority_population_percent', 'aus-1', 'denial_reason-1'],
      dtype='object')

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233144 entries, 0 to 233143
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   action_taken                       233144 non-null  int64  
 1   derived_race                       233144 non-null  object 
 2   derived_ethnicity                  233144 non-null  object 
 3   applicant_sex                      233144 non-null  int64  
 4   applicant_age                      233144 non-null  object 
 5   income                             226538 non-null  float64
 6   debt_to_income_ratio               100684 non-null  float64
 7   applicant_credit_score_type        233144 non-null  int64  
 8   loan_amount                        233144 non-null  float64
 9   loan_to_value_ratio                224447 non-null  float64
 10  interest_rate                      176143 non-null  float64
 11  rate_spread                        1519

In [96]:
df_race.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200974 entries, 0 to 233143
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   action_taken                       200974 non-null  int64  
 1   derived_race                       200974 non-null  object 
 2   derived_ethnicity                  200974 non-null  object 
 3   applicant_sex                      200974 non-null  int64  
 4   applicant_age                      200974 non-null  object 
 5   income                             197187 non-null  float64
 6   debt_to_income_ratio               86777 non-null   float64
 7   applicant_credit_score_type        200974 non-null  int64  
 8   loan_amount                        200974 non-null  float64
 9   loan_to_value_ratio                193263 non-null  float64
 10  interest_rate                      151991 non-null  float64
 11  rate_spread                        130793 no

In [97]:
age_map = {'<25': 1, '25-34': 2, '35-54' : 3, '45-54' : 4, '55-64' : 5, '65-74' : 6, '>74' : 7,'8888' : 8}
df_race['age_categorical'] = df_race['applicant_age'].map(age_map)
# 

# Model 1:  logistic regression that predicts loan outcome

Creating binary accepted colum (From ERin/Ellie)

In [100]:
df_race['action_taken'].unique()

array([1, 3, 8, 2, 6, 7], dtype=int64)

In [101]:
selected_df = df_race.loc[(df_race["action_taken"] == 1) | (df_race["action_taken"] == 2) | (df_race["action_taken"] == 3) | (df_race["action_taken"] == 7)]
selected_df = selected_df.copy()
selected_df['binary_accepted'] = selected_df['action_taken'].apply(lambda x: 1 if x in [1,2] else 0)
# Step 2: Create binary_accepted column

In [102]:
print(selected_df['binary_accepted'].unique(), selected_df['action_taken'].unique())

[1 0] [1 3 2 7]


In [103]:
selected_df['action_taken'].value_counts(), selected_df['binary_accepted'].value_counts()

(action_taken
 1    140754
 3     48666
 2      6615
 7       178
 Name: count, dtype: int64,
 binary_accepted
 1    147369
 0     48844
 Name: count, dtype: int64)

In [104]:
selected_df[['binary_accepted', 'action_taken']]
# selected_df

,binary_accepted,action_taken
0,1,1
1,1,1
2,1,1
3,1,1
6,1,1
...,...,...
233138,1,1
233139,0,3
233140,1,1
233142,1,1


In [105]:
log_model = smf.logit(formula = 'binary_accepted ~ tract_minority_population_percent + binary_race + debt_to_income_ratio + income + property_value + applicant_sex + age_categorical + applicant_credit_score_type + loan_to_value_ratio + loan_type + loan_purpose + lien_status + occupancy_type',
                       data = selected_df 
            )

results = log_model.fit()
print(results.summary())

C:\Users\erin\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


Optimization terminated successfully.
         Current function value: 0.384431
         Iterations 15
                           Logit Regression Results                           
Dep. Variable:        binary_accepted   No. Observations:                63798
Model:                          Logit   Df Residuals:                    63784
Method:                           MLE   Df Model:                           13
Date:                Wed, 09 Apr 2025   Pseudo R-squ.:                 0.04255
Time:                        11:17:44   Log-Likelihood:                -24526.
converged:                       True   LL-Null:                       -25616.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             3.9853      0.165     2

C:\Users\erin\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


In [106]:
log_model = smf.logit(formula = 'binary_accepted ~  debt_to_income_ratio + income + property_value + applicant_credit_score_type + loan_to_value_ratio + loan_type + loan_purpose + lien_status + occupancy_type',
                       data = selected_df 
            )

results = log_model.fit()
print(results.summary())

C:\Users\erin\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


Optimization terminated successfully.
         Current function value: 0.387170
         Iterations 15
                           Logit Regression Results                           
Dep. Variable:        binary_accepted   No. Observations:                84726
Model:                          Logit   Df Residuals:                    84716
Method:                           MLE   Df Model:                            9
Date:                Wed, 09 Apr 2025   Pseudo R-squ.:                 0.03430
Time:                        11:17:46   Log-Likelihood:                -32803.
converged:                       True   LL-Null:                       -33969.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       4.3341      0.129     33.478      0.000  

C:\Users\erin\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


##### Submodel trained on denials

In [108]:
selected_df = df_race.loc[(df_race["action_taken"] == 3) | (df_race["action_taken"] == 2) | (df_race["action_taken"] == 7)]
selected_df = selected_df.copy()
selected_df['binary_accepted'] = selected_df['action_taken'].apply(lambda x: 1 if x in [1,2] else 0)
# Step 2: Create binary_accepted column
# df_race["binary_accepted"] = df_race["action_taken"].isin([1, 2])

In [109]:
log_model = smf.logit(formula = 'binary_accepted ~ tract_minority_population_percent + binary_race + debt_to_income_ratio + income + property_value + applicant_sex + age_categorical + applicant_credit_score_type + loan_to_value_ratio + loan_type + loan_purpose + lien_status + occupancy_type',
                       data = selected_df 
            )

results = log_model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.497354
         Iterations 16
                           Logit Regression Results                           
Dep. Variable:        binary_accepted   No. Observations:                11199
Model:                          Logit   Df Residuals:                    11185
Method:                           MLE   Df Model:                           13
Date:                Wed, 09 Apr 2025   Pseudo R-squ.:                 0.04025
Time:                        11:17:46   Log-Likelihood:                -5569.9
converged:                       True   LL-Null:                       -5803.4
Covariance Type:            nonrobust   LLR p-value:                 1.391e-91
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.3207      0.323      

C:\Users\erin\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\erin\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


## Model 2: loan amount (MLR)

In [111]:
import statsmodels.api as sm
df_race = df_race.dropna()

X = df_race[['tract_minority_population_percent', "binary_race", "debt_to_income_ratio", 'income', "property_value", 'applicant_sex',
             'age_categorical', 'applicant_credit_score_type', 'loan_to_value_ratio', 'loan_type', 'loan_purpose', 'lien_status', 
             'occupancy_type', 'aus-1']]

y = df_race['loan_amount']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            loan_amount   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.795
Method:                 Least Squares   F-statistic:                 1.383e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        11:17:47   Log-Likelihood:            -6.7282e+05
No. Observations:               50031   AIC:                         1.346e+06
Df Residuals:                   50016   BIC:                         1.346e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

## Model 3: interest rate (MLR), rate_spread

In [113]:
df_race.columns


Index(['action_taken', 'derived_race', 'derived_ethnicity', 'applicant_sex',
       'applicant_age', 'income', 'debt_to_income_ratio',
       'applicant_credit_score_type', 'loan_amount', 'loan_to_value_ratio',
       'interest_rate', 'rate_spread', 'loan_type', 'loan_purpose',
       'lien_status', 'property_value', 'occupancy_type',
       'tract_minority_population_percent', 'aus-1', 'denial_reason-1',
       'binary_race', 'binary_ethnicity', 'binary_denied', 'age_categorical'],
      dtype='object')

In [114]:

df_race = df_race.dropna()
X = df_race[['tract_minority_population_percent', "binary_race", "debt_to_income_ratio", 'income', "property_value", 'applicant_sex',
             'age_categorical', 'applicant_credit_score_type', 'loan_to_value_ratio', 'loan_type', 'loan_purpose', 'lien_status', 
             'occupancy_type', 'aus-1']]

y = df_race['interest_rate']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          interest_rate   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     432.8
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        11:17:47   Log-Likelihood:                -88591.
No. Observations:               50031   AIC:                         1.772e+05
Df Residuals:                   50016   BIC:                         1.773e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [115]:
df_race.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50031 entries, 0 to 233140
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   action_taken                       50031 non-null  int64  
 1   derived_race                       50031 non-null  object 
 2   derived_ethnicity                  50031 non-null  object 
 3   applicant_sex                      50031 non-null  int64  
 4   applicant_age                      50031 non-null  object 
 5   income                             50031 non-null  float64
 6   debt_to_income_ratio               50031 non-null  float64
 7   applicant_credit_score_type        50031 non-null  int64  
 8   loan_amount                        50031 non-null  float64
 9   loan_to_value_ratio                50031 non-null  float64
 10  interest_rate                      50031 non-null  float64
 11  rate_spread                        50031 non-null  object 

In [116]:
df_race = df_race[df_race['rate_spread'] != 'Exempt']
df_race['rate_spread'] = pd.to_numeric(df_race['rate_spread'])

In [117]:

df_race = df_race.dropna()
X = df_race[['tract_minority_population_percent', "binary_race", "debt_to_income_ratio", 'income', "property_value", 'applicant_sex',
             'age_categorical', 'applicant_credit_score_type', 'loan_to_value_ratio', 'loan_type', 'loan_purpose', 'lien_status', 
             'occupancy_type', 'aus-1']]

y = df_race['rate_spread']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            rate_spread   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     161.2
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        11:17:47   Log-Likelihood:                -89164.
No. Observations:               50030   AIC:                         1.784e+05
Df Residuals:                   50015   BIC:                         1.785e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

# Recreation of model 
Train a model based on outcome (denial vs acceptance) using similar predictors
Look for bias in the outcomes of the model -> conclusion about biased data creating biased models in future 

In [119]:
from sklearn.model_selection import train_test_split

# select columns that were explicitly approved or denied
filtered_df = df.loc[(df["action_taken"] == 1) | (df["action_taken"] == 2) | (df["action_taken"] == 7) | (df["action_taken"] == 3)]

# remove na values
filtered_df = filtered_df.dropna(subset=["debt_to_income_ratio","income","loan_to_value_ratio","loan_amount","property_value"])

# create binary accepted column
filtered_df["binary_accepted"] = True
filtered_df.loc[filtered_df["action_taken"] == 7, 'binary_accepted'] = False
filtered_df.loc[filtered_df["action_taken"] == 3, 'binary_accepted'] = False
counts = filtered_df['binary_accepted'].value_counts()

# create binary race column - 1 for White
df_slice = filtered_df.loc[(filtered_df['derived_race'] == "Black or African American") | (filtered_df['derived_race'] == "White")]
filtered_df = df_slice.copy()
filtered_df['binary_race'] = 0
filtered_df.loc[filtered_df['derived_race'] == "White", 'binary_race'] = 1

# split data

X = filtered_df[["debt_to_income_ratio","income","loan_to_value_ratio","loan_amount","property_value"]]
y = filtered_df["binary_accepted"]

train, test = train_test_split(filtered_df, test_size=0.2)
X_train, X_test = train[["debt_to_income_ratio","income","loan_to_value_ratio","loan_amount","property_value"]], \
test[["debt_to_income_ratio","income","loan_to_value_ratio","loan_amount","property_value"]] 
y_train, y_test = train["binary_accepted"], test["binary_accepted"] 

In [120]:
filtered_df['binary_accepted'].value_counts(), filtered_df['action_taken'].value_counts()

(binary_accepted
 True     73052
 False    11674
 Name: count, dtype: int64,
 action_taken
 1    69923
 3    11612
 2     3129
 7       62
 Name: count, dtype: int64)

In [121]:
filtered_df[filtered_df['binary_accepted'] == False]

,action_taken,derived_race,derived_ethnicity,applicant_sex,applicant_age,income,debt_to_income_ratio,applicant_credit_score_type,loan_amount,loan_to_value_ratio,...,loan_type,loan_purpose,lien_status,property_value,occupancy_type,tract_minority_population_percent,aus-1,denial_reason-1,binary_accepted,binary_race
186,3,White,Not Hispanic or Latino,2,>74,35.0,49.0,11,155000.0,18.750,...,1,32,1,805000.0,1,41.18,6,4,False,1
228,3,White,Not Hispanic or Latino,1,35-44,181.0,44.0,9,205000.0,89.052,...,1,4,2,715000.0,1,17.34,6,3,False,1
277,3,White,Not Hispanic or Latino,1,55-64,102.0,42.0,2,15000.0,69.499,...,1,32,2,195000.0,1,6.16,6,9,False,1
285,3,White,Not Hispanic or Latino,1,45-54,131.0,48.0,2,25000.0,70.664,...,1,2,2,145000.0,1,6.67,6,9,False,1
323,3,White,Not Hispanic or Latino,2,55-64,292.0,41.0,3,435000.0,95.000,...,1,1,1,455000.0,1,11.40,1,9,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233096,3,White,Not Hispanic or Latino,2,65-74,91.0,44.0,9,85000.0,62.600,...,1,31,2,335000.0,1,8.02,6,3,False,1
233100,3,White,Not Hispanic or Latino,2,35-44,118.0,39.0,3,65000.0,81.471,...,1,4,2,225000.0,1,12.19,6,3,False,1
233106,3,White,Not Hispanic or Latino,2,35-44,132.0,39.0,9,25000.0,88.336,...,1,4,2,145000.0,1,7.11,6,3,False,1
233117,3,White,Hispanic or Latino,1,55-64,271.0,39.0,2,175000.0,89.611,...,1,2,2,955000.0,1,37.53,6,3,False,1


Model 1: Logistic Regression

In [123]:
# logistic regression
from sklearn import metrics # check model accuracy

lr = sk.linear_model.LogisticRegression()
lr = LogisticRegression(max_iter=3000)

lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
acc_lr = sk.metrics.accuracy_score(y_pred,y_test)
print(acc_lr)

0.8649238758409064


Model 2: Decision Tree

In [125]:
# decision tree model
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics # check model accuracy

dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
acc_dt = metrics.accuracy_score(y_pred,y_test)
print(acc_dt)

0.7850820252566978


Model 3: K-Nearest Neighbors

In [127]:
# knn
from sklearn.neighbors import KNeighborsClassifier # KNN

knc = KNeighborsClassifier(n_neighbors=3) # for k=3
knc.fit(X_train,y_train)
y_pred = knc.predict(X_test)
acc_knn = sk.metrics.accuracy_score(y_pred,y_test)
print(acc_knn)

0.8358904756284669


In [128]:
# plot overall accuracy comparison between models
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree','K-Nearest Neighbours'],
    'Score': [acc_lr, acc_dt, acc_knn]})
models.sort_values(by='Score', ascending=False)

,Model,Score
0,Logistic Regression,0.864924
2,K-Nearest Neighbours,0.835890
1,Decision Tree,0.785082


Look for algorithmic bias by comparing accuracy rates across races.

In [130]:
# split data and get predictions between men and women
x_var = ["debt_to_income_ratio","income","loan_to_value_ratio","loan_amount","property_value"]

test_b = test.loc[test['derived_race'] == "Black or African American"]
test_w = test.loc[test['derived_race'] == "White"]

X_test_b, X_test_w = test_b[x_var], test_w[x_var]
y_test_b, y_test_w = test_b['binary_accepted'], test_w['binary_accepted']

y_pred_b = knc.predict(X_test_b)
y_pred_w = knc.predict(X_test_w)

In [131]:
# report accuracy
acc_log_b = sk.metrics.accuracy_score(y_pred_b, y_test_b)
acc_log_w = sk.metrics.accuracy_score(y_pred_w, y_test_w)
print('Black or African American: ', acc_log_b)
print('White: ', acc_log_w)

Black or African American:  0.7606659729448492
White:  0.845513844515442


Examine selection, false positive, and false negative rates when model takes race into account

In [133]:
import fairlearn
from fairlearn.metrics import demographic_parity_ratio, MetricFrame, selection_rate, false_positive_rate, false_negative_rate
from sklearn import metrics
from sklearn.metrics import accuracy_score

X_train, X_test = train[["debt_to_income_ratio","income","loan_to_value_ratio","loan_amount","property_value", "binary_race"]], \
test[["debt_to_income_ratio","income","loan_to_value_ratio","loan_amount","property_value", "binary_race"]] 

dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
acc_dt = metrics.accuracy_score(y_pred,y_test)

In [134]:
metrics = {
    'accuracy' : accuracy_score,
    'selection rate' : selection_rate,
    'FPR' : false_positive_rate,
    'FNR' : false_negative_rate
}

# sensitive_features is an array (or series)
mf = MetricFrame(metrics = metrics, y_true=y_test, y_pred=y_pred, sensitive_features=X_test['binary_race'])
mf.by_group

,accuracy,selection rate,FPR,FNR
binary_race,,,,
0,0.693548,0.772633,0.684466,0.203311
1,0.797191,0.855365,0.733191,0.127224
